In [ ]:
%tensorflow_version 2.x
!pip install 'Keras==2.4.3'
#!pip install 'keras==2.1.6'


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
%tensorflow_version 2.x

Mounted at /content/drive


In [ ]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential ,load_model
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.metrics import Accuracy
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [ ]:
filename = "/content/drive/MyDrive/MSVD/youtube_video_to_id_mapping.txt"
file = open(filename, 'r')
text1 = file.read()
file.close()
#print(text1)
videotoid=dict()
for line in text1.split('\n'):
    if len(line)<2:
        continue
    list=line.split(' ')
    #print(list[0],list[1])
    videotoid[list[0]+'.npy']=list[1]
print(videotoid)

{'-4wsuPCjDBc_5_15.npy': 'vid1', '-7KMZQEsJW4_205_208.npy': 'vid2', '-8y1Q0rA3n8_108_115.npy': 'vid3', '-8y1Q0rA3n8_95_102.npy': 'vid4', '-9CUm-2cui8_39_44.npy': 'vid5', '-AwoiGR6c8M_10_14.npy': 'vid6', '-Cv5LsqKUXc_17_25.npy': 'vid7', '-Cv5LsqKUXc_71_76.npy': 'vid8', '-DKuLXYoY3g_14_20.npy': 'vid9', '-DRy7rBg0IQ_31_37.npy': 'vid10', '-FugkxLmGO4_5_16.npy': 'vid11', '-Ms9tsWmhyU_80_95.npy': 'vid12', '-YI0cxuNcq8_262_272.npy': 'vid13', '-_aaMGK6GGw_57_61.npy': 'vid14', '-_hbPLsZvvo_172_179.npy': 'vid15', '-_hbPLsZvvo_18_25.npy': 'vid16', '-_hbPLsZvvo_19_25.npy': 'vid17', '-_hbPLsZvvo_19_26.npy': 'vid18', '-_hbPLsZvvo_211_219.npy': 'vid19', '-_hbPLsZvvo_269_275.npy': 'vid20', '-_hbPLsZvvo_288_305.npy': 'vid21', '-_hbPLsZvvo_323_328.npy': 'vid22', '-_hbPLsZvvo_43_55.npy': 'vid23', '-_hbPLsZvvo_49_55.npy': 'vid24', '-_hbPLsZvvo_5_8.npy': 'vid25', '-bjOB4zS0uE_100_105.npy': 'vid26', '-dm-ds5rRaM_44_52.npy': 'vid27', '-joBOHlg5J0_72_80.npy': 'vid28', '-mAoVOhKy0c_4_9.npy': 'vid29', '-pUwIypk

In [ ]:
# load doc into memorymypath='/content/drive/MyDrive/MSVD/i3d_features'
del list
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

filename = "/content/drive/MyDrive/MSVD/sents_train_lc_nopunc.txt"
# load descriptions
doc = load_doc(filename)
print(doc[1100:2100])

ing two dried sunflowers is talking while dusting off the center portion of one flower
vid2	a man is brushing dirt off a sunflower
vid2	a man is brushing or patting the top of a sunflower with his hand
vid2	a man is holding dead sunflowers
vid2	a man is holding huge sunflowers
vid2	a man is holding two sunflowers
vid2	a man is knocking seeds out of a sunflower
vid2	a man is rubbing the head of a sunflower with his hand
vid2	a man is showing sunflowers
vid2	a man is tapping a sunflower
vid2	a man rubs a sunflower
vid2	a man rubs a sunflower
vid2	a man showed a sunflower head
vid2	someone is holding sunflowers
vid2	the man exhibited a sunflower head
vid2	the man is brushing dry sunflowers
vid2	virginia farm bureau in the garden sunflowers
vid2	a guy is holding the sunflower in his hand
vid2	a man is clearing sunflower
vid2	a man is holding two sunflowers
vid2	a man is picking some sunflowers
vid2	a man show flower in the garden
vid2	a man showing a sunflower
vid2	one man taking sunflower

In [ ]:
# parse descriptions
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# extract filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
	return mapping

descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

Loaded: 1200 


In [ ]:
list(descriptions.keys())[:10]

['vid1',
 'vid2',
 'vid3',
 'vid4',
 'vid5',
 'vid6',
 'vid7',
 'vid8',
 'vid9',
 'vid10']

In [ ]:
len(descriptions['vid4'])

27

In [ ]:
# clean descriptions
clean_descriptions(descriptions)

NameError: ignored

In [ ]:
descriptions['vid3']

['a man slices through a two liter plastic bottle of soda pop with a sword',
 'a man slices a soda bottle with a sword',
 'a man slices a soda bottle with a sword',
 'a man uses a sword to cut a two liter plastic bottle in half',
 'a man is broken a bottle by a sword',
 'a man sliced a plastic bottle with a sword',
 'a man is cutting a bottle with a sword',
 'a man is cutting a bottle of water with a sword',
 'someone slices a bottle with a sword',
 'a person slices through a bottle with a sword',
 'the ma sliced through a plastic bottle',
 'the man used a sword to slice a plastic bottle',
 'a man slices the top of a plastic bottle of water off with a sword',
 'a man breaks a bottle with a sword',
 'the man sliced through a soda bottle with a sword',
 'the man cut the bottle with the sword',
 'a person is break out the bottle',
 'a man cuts a bottle with a sword',
 'i love the cold steel blades',
 'a person cutting a botal',
 'the person is doing the something',
 'a man is cutting a bo

In [ ]:
descriptions['vid5']

['a woman puts four okra in a pan of boiling water',
 'a woman is boiling peppers',
 'a person is boiling chile peppers',
 'someone is cooking okra in a pan',
 'a woman is boiling finger in the water',
 'a woman is adding cut green beans to boiling water',
 'a woman is boiling some vegetables',
 'okra is cooking in boiling water',
 'someone is boiling some peppers',
 'a woman is adding vegetables to a pot',
 'the lady added the pea pods to the boiling water',
 'the woman is boiling okra',
 'a woman is boiling okra',
 'a woman dumps vegetables into boiling water',
 'ocra is boiling in a pot of water',
 'someone is boiling okra in a pot',
 'the lady put the bean pods in the boiling water',
 'there is a preparing curry',
 'a lady is stirring the lady finger which is in the boiling water',
 'someone is boiling peppers',
 'the person is the boiling the ladiesfinger',
 'a woman is putting some okra into a pot',
 'a chef is boiling vegetables in water',
 'someone boiling lady finger in frying

In [ ]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# build a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 9462


In [ ]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_descriptions(descriptions, '/content/drive/MyDrive/resume/descriptions.txt')

In [ ]:
# load names of videoids[vid1 ,vid2 ...] 


def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split()[0]
		dataset.append(identifier)
	return set(dataset)

# load training dataset (6K)
filename = '/content/drive/MyDrive/MSVD/sents_train_lc_nopunc.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))

Dataset: 1200


In [ ]:
print(train)
len(train)

{'vid50', 'vid462', 'vid184', 'vid533', 'vid291', 'vid775', 'vid770', 'vid280', 'vid546', 'vid405', 'vid562', 'vid1024', 'vid1167', 'vid524', 'vid694', 'vid580', 'vid240', 'vid450', 'vid590', 'vid514', 'vid547', 'vid516', 'vid565', 'vid890', 'vid32', 'vid244', 'vid15', 'vid849', 'vid299', 'vid1011', 'vid912', 'vid134', 'vid364', 'vid366', 'vid491', 'vid1', 'vid70', 'vid75', 'vid587', 'vid815', 'vid672', 'vid102', 'vid202', 'vid319', 'vid545', 'vid141', 'vid640', 'vid776', 'vid920', 'vid179', 'vid677', 'vid853', 'vid1128', 'vid372', 'vid1003', 'vid959', 'vid749', 'vid1170', 'vid200', 'vid263', 'vid612', 'vid521', 'vid206', 'vid868', 'vid798', 'vid41', 'vid557', 'vid731', 'vid1057', 'vid235', 'vid889', 'vid1193', 'vid1028', 'vid38', 'vid493', 'vid1106', 'vid636', 'vid1156', 'vid561', 'vid1159', 'vid991', 'vid40', 'vid751', 'vid980', 'vid831', 'vid1058', 'vid926', 'vid230', 'vid1129', 'vid381', 'vid799', 'vid725', 'vid223', 'vid387', 'vid739', 'vid841', 'vid59', 'vid269', 'vid418', 'vid35

1200

In [ ]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# descriptions
train_descriptions = load_clean_descriptions('/content/drive/MyDrive/resume/descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))

Descriptions: train=1200


In [ ]:
print(train_descriptions)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#Load videoid to video mapping
doc=load_doc('/content/drive/MyDrive/MSVD/youtube_video_to_id_mapping.txt')
vidid_to_vidname={}
vidname_to_vidid={}
lines=doc.split('\n')
for line in lines:
  x=line.split(' ')
  if len(x) ==2:
    vid_name=x[0]
    vid_id=x[1]
    vidid_to_vidname[vid_id]=vid_name
    vidname_to_vidid[vid_name]=vid_id
print(len(vidid_to_vidname))

1970


In [ ]:
#vidid_to_vidname

In [ ]:
# Create a list of all the training captions
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
len(all_train_captions)

48774

In [ ]:
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 8
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

preprocessed words 9464 -> 2264


In [ ]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1
    

In [ ]:
vocab_size = len(ixtoword) + 1 # one for appended 0's
vocab_size

2265

In [ ]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

Description Length: 47


In [ ]:
def get_feat_by_vidname(name):
              vid_feat_file_name='i3d_resnet50_v1_kinetics400_'+name+'.avi_feat.npy'
              vid_feat_complete_path='/content/drive/MyDrive/MSVD/i3d_features/'+vid_feat_file_name
              photo = np.load(vid_feat_complete_path)
              photo.reshape((1,2048))
              return photo


In [ ]:
def get_feat_by_vidname(name):
              vid_feat_file_name='i3d_resnet50_v1_kinetics400_'+name+'.avi_feat.npy'
              vid_feat_complete_path='/content/drive/MyDrive/MSVD/i3d_features/'+vid_feat_file_name
              photo = np.load(vid_feat_complete_path)
              photo.reshape((1,2048))
              return photo

In [ ]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator1(descriptions, wordtoix, max_length, num_photos_per_batch):
    X1, X2 , y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            try:
              # retrieve the photo feature
              vid_feat_file_name='i3d_resnet50_v1_kinetics400_'+vidid_to_vidname[key]+'.avi_feat.npy'
              vid_feat_complete_path='/content/drive/MyDrive/MSVD/i3d_features/'+vid_feat_file_name
              photo = np.load(vid_feat_complete_path)
              photo=np.reshape(photo, photo.shape[1])       
              #print("no error")
              n+=1
              for desc in desc_list:
                  # encode the sequence
                  seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                  # split one sequence into multiple X, y pairs
                  for i in range(1, len(seq)):
                      # split into input and output pair
                      in_seq, out_seq = seq[:i], seq[i]
                      # pad input sequence
                      in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                      # encode output sequence
                      out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                      # store
                      X1.append(photo)
                      X2.append(in_seq)
                      #X3.append(photo2)
                      y.append(out_seq)
              # yield the batch data
              if n==num_photos_per_batch:
                  # print([[array(X1), array(X2)], array(y)])
                  yield [array(X1), array(X2),array(X3)],array(y)
                  X1, X2, y = list(),list(), list()
                  n=0
            except:
              continue


In [ ]:
vidid_to_vidname["vid1"]

'-4wsuPCjDBc_5_15'

In [ ]:
m = np.matrix([[1,2], [3,4]])
print(m.shape)
ls=m.flatten()
print(ls.shape[0])
ls


(2, 2)
1


matrix([[1, 2, 3, 4]])

In [ ]:
mx=0
for key, desc_list in descriptions.items():
    ans=map_maskrcnn[key].shape[0]
    #print(ans.shape[0],ans.shape[1])
    if mx<ans:
        mx=ans
mx

NameError: ignored

In [ ]:
map_i3d=dict()
map_maskrcnn=dict()
for key, desc_list in descriptions.items():

    vid_feat_file_name='i3d_resnet50_v1_kinetics400_'+vidid_to_vidname[key]+'.avi_feat.npy'
    vid_feat_complete_path='/content/drive/MyDrive/MSVD/i3d_features/'+vid_feat_file_name
    photo=np.load(vid_feat_complete_path)
    photo=np.reshape(photo, photo.shape[1])
    map_i3d[key]= photo

    maskrcnn_file_name=vidid_to_vidname[key]+'.npy'
    maskrcnn_complete_path='/content/drive/MyDrive/MASK_RCNN_FEATURES/'+ maskrcnn_file_name
    photo2=np.load(maskrcnn_complete_path)
    photo2=photo2.flatten()
    map_maskrcnn[key]= photo2

In [ ]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, wordtoix, max_length, num_photos_per_batch):
    X1, X2, X3 , y = list(), list(),list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            try:
              # retrieve the photo feature
              '''vid_feat_file_name='i3d_resnet50_v1_kinetics400_'+vidid_to_vidname[key]+'.avi_feat.npy'
              vid_feat_complete_path='/content/drive/MyDrive/MSVD/i3d_features/'+vid_feat_file_name
              photo = np.load(vid_feat_complete_path)
              photo=np.reshape(photo, photo.shape[1])
              maskrcnn_file_name=vidid_to_vidname[key]+'.npy'
              maskrcnn_complete_path='/content/drive/MyDrive/MASK_RCNN_FEATURES/'+ maskrcnn_file_name
              photo2 = np.load(maskrcnn_complete_path)
              print(key,maskrcnn_file_name,vidid_to_vidname[key])
              photo2=np.reshape(photo2,photo2.shape[1])
              #print(photo.shape)
              #print(photo2.shape)  
              print(photo2)  
              print(photo)'''
              photo=map_i3d[key]
              photo2=map_maskrcnn[key]
              photo2.resize(518400)
              #print(photo.shape)
              #print(photo2.shape)  
              #print("no error")
              n+=1
              #print(n)
              if n==num_photos_per_batch+1:
                  # print([[array(X1), array(X2)], array(y)])
                  yield [array(X1), array(X2),array(X3)],array(y)
                  X1, X2, X3, y = list(), list(),list(), list()
                  n=0
              for desc in desc_list:
                  # encode the sequence
                  seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                  # split one sequence into multiple X, y pairs
                  for i in range(1, len(seq)):
                      # split into input and output pair
                      in_seq, out_seq = seq[:i], seq[i]
                      # pad input sequence
                      in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                      # encode output sequence
                      out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                      # store
                      X1.append(photo)
                      X2.append(in_seq)
                      X3.append(photo2)
                      y.append(out_seq)
              if n==num_photos_per_batch:
                  # print([[array(X1), array(X2)], array(y)])
                  yield [array(X1), array(X2),array(X3)],array(y)
                  X1, X2, X3, y = list(), list(),list(), list()
                  n=0
            except:
              continue


In [ ]:
b = np.array([[0 ,1, 2, 3, 4, 5, 6],[0 ,1, 2, 3, 4, 5, 6]])
b=b.flatten()
b.resize(20)
b.shape
#b.resize(20) # new_shape parameter doesn't have to be a tuple
#b

(20,)

In [ ]:
number_photos_per_bath=1
generator = data_generator(train_descriptions, wordtoix, max_length, number_photos_per_bath)
inputs, outputs = next(generator)
print(inputs[0].shape)
print(inputs[1].shape)
print(inputs[2].shape)
print(outputs.shape)


(189, 2048)
(189, 47)
(189, 518400)
(189, 2265)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load Glove vectors
embeddings_index = {} # empty dictionary
f = open('/content/drive/MyDrive/resume/glove.6B.300d.txt', encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
embedding_dim = 300
# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(2265, 300)

In [ ]:

inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

inputs3 = Input(shape=(518400,))
fe3 = Dropout(0.5)(inputs3)
fe4 = Dense(256, activation='relu')(fe3)

decoder1 = add([fe2,se3,fe4])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2,inputs3], outputs=outputs)

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 47)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 47, 300)      679500      input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 518400)]     0                                            
_______________________________________________________________________________________

In [ ]:
model.layers[2]

In [ ]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 10
number_photos_per_bath = 2
steps = len(train_descriptions)//number_photos_per_bath

In [ ]:
generator = data_generator(train_descriptions, wordtoix, max_length, number_photos_per_bath)
inputs, outputs = next(generator)
inputs, outputs = next(generator)
print(inputs[0].shape)
print(inputs[1].shape)
print(inputs[2].shape)
print(outputs.shape)

(513, 2048)
(513, 47)
(513, 518400)
(513, 2265)


Exception ignored in: <generator object data_generator at 0x7f7dcc36dc50>
RuntimeError: generator ignored GeneratorExit


In [ ]:
import numpy
for i in range(epochs):
    print(i)
    generator = data_generator(train_descriptions, wordtoix, max_length, number_photos_per_bath)
    print(generator)
    print('return from gen')
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    print('model ' + str(i) + " done")
    model.save('/content/drive/Shareddrives/GRAPH_VC/model_weights/model_' + str(i) + '.h5')

0
<generator object data_generator at 0x7f7dcc36df10>
return from gen
Instructions for updating:
Please use Model.fit, which supports generators.


Exception ignored in: <generator object data_generator at 0x7f7e20073048>
RuntimeError: generator ignored GeneratorExit


600/600 [==============================] - 229s 382ms/step - loss: 5.3020 - accuracy: 0.2798
model 0 done
1
<generator object data_generator at 0x7f7e201cf9e8>
return from gen
600/600 [==============================] - 231s 384ms/step - loss: 2.0654 - accuracy: 0.5122
model 1 done
2
<generator object data_generator at 0x7f7d3b5603b8>
return from gen
600/600 [==============================] - 231s 384ms/step - loss: 1.6443 - accuracy: 0.5743
model 2 done
3
<generator object data_generator at 0x7f7d3b20af10>
return from gen
600/600 [==============================] - 234s 390ms/step - loss: 1.3194 - accuracy: 0.6295
model 3 done
4
<generator object data_generator at 0x7f7dcc26d5c8>
return from gen
600/600 [==============================] - 233s 389ms/step - loss: 1.1384 - accuracy: 0.6743
model 4 done
5
<generator object data_generator at 0x7f7e298c1678>
return from gen
600/600 [==============================] - 233s 388ms/step - loss: 0.9777 - accuracy: 0.7124
model 5 done
6
<generator o

In [ ]:
print(train_descriptions, wordtoix, max_length, number_photos_per_bath)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#mask rcnn features
MASK_RCNN_FEATURES=dict()
import numpy as np
from os import listdir
from os.path import isfile, join
mypath='/content/drive/MyDrive/MASK_RCNN_FEATURES'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for a in onlyfiles:
    pp=np.load(join(mypath,a))
    MASK_RCNN_FEATURES[videotoid[a]]=pp
     # print(pp.shape)
    #pp=pp.reshape((1,2048))
#print(MASK_RCNN_FEATURES)




In [ ]:
#model=load_model('/content/drive/MyDrive/resume/Weights9.h5')
model=load_model('/content/drive/Shareddrives/GRAPH_VC/model_weights/model_9.h5')

In [ ]:
def greedySearch(photo1,photo2):
    in_text = 'startseq'
    #np.resize(photo2,518400)
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo1,sequence,photo2], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [ ]:
from os import listdir
from os.path import isfile, join
mypath1='/content/drive/MyDrive/MSVD/i3d_features'
mypath2='/content/drive/MyDrive/MASK_RCNN_FEATURES'
onlyfilisfilees1 = [f for f in listdir(mypath1) if isfile(join(mypath1, f))]
onlyfiles2 = [f for f in listdir(mypath2) if isfile(join(mypath2, f))]
onlyfiles=[[onlyfiles1,onlyfiles2]]

print(onlyfiles[0][1])

['1qxMJLkHDLw_0_9.npy', 'j2Dhf-xFUxU_20_29.npy', 'zv2RIbUsnSw_335_341.npy', '6r5DmFa5ZR8_51_58.npy', 'j7xz1nos-xc_10_20.npy', 'FeTaKOPVaJg_60_70.npy', 'uVPnDJKt1M0_0_6.npy', 'n_Z0-giaspE_168_193.npy', 'MWvCcwTw7Ac_78_86.npy', '2FLsMPsywRc_45_63.npy', 'hJFBXHtxKIc_286_291.npy', 'q3I3R_gqy8M_34_37.npy', 'HM-ZDoRWiH4_0_5.npy', 'BVjvRpmHg0w_231_250.npy', 'NUYu9c9XsgY_7_21.npy', 'YS1mzzhmWWA_9_16.npy', 'ACOmKiJDkA4_75_81.npy', 'ZDoy9S82ETA_5_15.npy', 'uxEhH6MPH28_69_85.npy', 'F3pmw2ga244_59_65.npy', 'm7x8uIdg2XU_67_73.npy', 'qIk_Dz5XE5E_104_109.npy', '2YhDTpzxd3c_166_170.npy', '6t0BpjwYKco_230_240.npy', '0lh_UWF9ZP4_103_110.npy', 'x68Djm_Q0GA_0_10.npy', 'lc9bA-hvqHU_1_6.npy', 'R2DvpPTfl-E_20_25.npy', 'uO7Eysrs8_k_106_114.npy', 'X6uJyuD_Zso_3_17.npy', 'zYcY4mjLpxU_104_118.npy', 'g36ho6UrBz0_5_20.npy', 'ZbzDGXEwtGc_6_15.npy', 'buJ5HDCinrM_150_166.npy', 'mv89psg6zh4_33_46.npy', '04Gt01vatkk_248_265.npy', 'jbzaMtPYtl8_48_58.npy', '6q1dX6thX3E_286_295.npy', 'M_TP6UTrh30_343_358.npy', 'n016q1w8Q3

In [ ]:
n=0
mypath1='/content/drive/MyDrive/MSVD/i3d_features'
mypath2='/content/drive/MyDrive/MASK_RCNN_FEATURES'
for a,b in zip(onlyfiles1,onlyfiles2):
  pp1=np.load(join(mypath1,a))
  pp2=np.load(join(mypath2,b))
  # print(pp.shape)
  #pp1=np.reshape(pp1, pp1.shape[1])
  pp1=pp1.reshape((1,2048))
  pp2=pp2.flatten()
  pp2.resize(518400)
  pp2=pp2.reshape((1,518400))
  #pp2=pp2.reshape((1,2048))
  print(pp1.shape,pp2.shape)
  print(pp2)
  print("Greedy:",greedySearch(pp1,pp2))
  print(a)
  n+=1
  if n==5:
    break

(1, 2048) (1, 518400)
[[0 0 0 ... 0 0 0]]
Greedy: a man is holding a woman are dancing
i3d_resnet50_v1_kinetics400_1qxMJLkHDLw_0_9.avi_feat.npy
(1, 2048) (1, 518400)
[[0. 0. 0. ... 0. 0. 0.]]
Greedy: a man is slicing a potato
i3d_resnet50_v1_kinetics400_j2Dhf-xFUxU_20_29.avi_feat.npy
(1, 2048) (1, 518400)
[[0 0 0 ... 0 0 0]]
Greedy: a man is slicing a hallway
i3d_resnet50_v1_kinetics400_zv2RIbUsnSw_335_341.avi_feat.npy
(1, 2048) (1, 518400)
[[0 0 0 ... 0 0 0]]
Greedy: a girl is playing with a machine
i3d_resnet50_v1_kinetics400_6r5DmFa5ZR8_51_58.avi_feat.npy
(1, 2048) (1, 518400)
[[0 0 0 ... 0 0 0]]
Greedy: a man is playing a flute
i3d_resnet50_v1_kinetics400_j7xz1nos-xc_10_20.avi_feat.npy


Greedy: a man is singing
i3d_resnet50_v1_kinetics400_1qxMJLkHDLw_0_9.avi_feat.npy
Greedy: a woman is slicing a potato
i3d_resnet50_v1_kinetics400_j2Dhf-xFUxU_20_29.avi_feat.npy
Greedy: a man is talking on the phone
i3d_resnet50_v1_kinetics400_zv2RIbUsnSw_335_341.avi_feat.npy
Greedy: a baby is eating
i3d_resnet50_v1_kinetics400_6r5DmFa5ZR8_51_58.avi_feat.npy
Greedy: a man is playing a piano
i3d_resnet50_v1_kinetics400_j7xz1nos-xc_10_20.avi_feat.npy


In [ ]:
from nltk.translate.bleu_score import corpus_bleu 
!pip install --upgrade nltk
import nltk
nltk.download('wordnet')
# from nltk.translate import meteor_score

     |████████████████████████████████| 1.4MB 19.3MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434676 sha256=89ec6dadbe85d41482f13775b6d9e0411ae1c6f6cfdc1b6aa0952bf5420294a5
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
#BElU
actual=[]
predicted = []
for key, desc_list in train_descriptions.items():
      pp1=map_i3d[key]
      pp2=map_maskrcnn[key]
      pp1=pp1.reshape((1,2048))
      pp2=np.resize(pp2,518400)
      pp2=np.reshape(pp2,(1,518400))
      yhat = greedySearch(pp1,pp2)
      # store actual and predicted
      references = descriptions[key]
      ref = [d.split() for d in references]
      actual.append(ref)
      predicted.append(yhat.split())

In [ ]:
print(len(predicted))

1200


In [ ]:
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

BLEU-1: 0.660626
BLEU-2: 0.482434
BLEU-3: 0.414540
BLEU-4: 0.279653


BLEU-1: 0.795119
BLEU-2: 0.678442
BLEU-3: 0.618897
BLEU-4: 0.490904

In [ ]:
!pip install pycocoevalcap

     |████████████████████████████████| 104.3MB 61kB/s 


In [ ]:
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider

In [ ]:
class COCOEvalCap:
    def __init__(self,images,gts,res):
        self.evalImgs = []
        self.eval = {}
        self.imgToEval = {}
        self.params = {'image_id': images}
        self.gts = gts
        self.res = res

    def evaluate(self):
        imgIds = self.params['image_id']
        gts = self.gts
        res = self.res

        # =================================================
        # Set up scorers
        # =================================================
        print ('tokenization...')
        tokenizer = PTBTokenizer()
        gts  = tokenizer.tokenize(gts)
        res = tokenizer.tokenize(res)

        # =================================================
        # Set up scorers
        # =================================================
        print ('setting up scorers...')
        scorers = [
            (Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
            (Meteor(),"METEOR"),
            (Rouge(), "ROUGE_L"),
            (Cider(), "CIDEr")
        ]

        # =================================================
        # Compute scores
        # =================================================
        eval = {}
        for scorer, method in scorers:
            print ('computing %s score...'%(scorer.method()))
            score, scores = scorer.compute_score(gts, res)
            if type(method) == list:
                for sc, scs, m in zip(score, scores, method):
                    self.setEval(sc, m)
                    self.setImgToEvalImgs(scs, imgIds, m)
                    print ("%s: %0.3f"%(m, sc))
            else:
                self.setEval(score, method)
                self.setImgToEvalImgs(scores, imgIds, method)
                print ("%s: %0.3f"%(method, score))
        self.setEvalImgs()

    def setEval(self, score, method):
        self.eval[method] = score

    def setImgToEvalImgs(self, scores, imgIds, method):
        for imgId, score in zip(imgIds, scores):
            if not imgId in self.imgToEval:
                self.imgToEval[imgId] = {}
                self.imgToEval[imgId]["image_id"] = imgId
            self.imgToEval[imgId][method] = score

    def setEvalImgs(self):
        self.evalImgs = [eval for imgId, eval in self.imgToEval.items()]

def calculate_metrics(rng,datasetGTS,datasetRES):
    imgIds = rng
    gts = {}
    res = {}

    imgToAnnsGTS = {ann['image_id']: [] for ann in datasetGTS['annotations']}
    for ann in datasetGTS['annotations']:
        imgToAnnsGTS[ann['image_id']] += [ann]

    imgToAnnsRES = {ann['image_id']: [] for ann in datasetRES['annotations']}
    for ann in datasetRES['annotations']:
        imgToAnnsRES[ann['image_id']] += [ann]

    for imgId in imgIds:
        gts[imgId] = imgToAnnsGTS[imgId]
        res[imgId] = imgToAnnsRES[imgId]

    evalObj = COCOEvalCap(imgIds,gts,res)
    evalObj.evaluate()
    return evalObj.eval

if __name__ == '__main__':
    rng = range(2)
    datasetGTS = {
        'annotations': [{u'image_id': 0, u'caption': u'the man is running'},
                        {u'image_id': 1, u'caption': u'a woman is chopping cucumbers and carrots'}]
        }
    datasetRES = {
        'annotations': [{u'image_id': 0, u'caption': u'the man is playing a guitar'},
                        {u'image_id': 1, u'caption': u'a woman is cutting cucumbers and carrots'}]
        }
    print (calculate_metrics(rng,datasetGTS,datasetRES))

tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 13, 'reflen': 11, 'guess': [13, 11, 9, 7], 'correct': [9, 6, 3, 0]}
ratio: 1.181818181710744
Bleu_1: 0.692
Bleu_2: 0.615
Bleu_3: 0.501
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.434
computing Rouge score...
ROUGE_L: 0.740
computing CIDEr score...
CIDEr: 4.014
{'Bleu_1': 0.692307692254438, 'Bleu_2': 0.6145098677474599, 'Bleu_3': 0.5011627948387378, 'Bleu_4': 6.511928251713921e-05, 'METEOR': 0.43436118998539447, 'ROUGE_L': 0.7397959183673469, 'CIDEr': 4.014287395082455}


tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 13, 'reflen': 11, 'guess': [13, 11, 9, 7], 'correct': [9, 6, 3, 0]}
ratio: 1.181818181710744
Bleu_1: 0.692
Bleu_2: 0.615
Bleu_3: 0.501
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.434
computing Rouge score...
ROUGE_L: 0.740
computing CIDEr score...
CIDEr: 4.014
{'Bleu_1': 0.692307692254438, 'Bleu_2': 0.6145098677474599, 'Bleu_3': 0.5011627948387378, 'Bleu_4': 6.511928251713921e-05, 'METEOR': 0.43436118998539447, 'ROUGE_L': 0.7397959183673469, 'CIDEr': 4.014287395082455}

In [ ]:
print(actual)
print(predicted)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
num_predictions = len(predicted)

rng = range(num_predictions)

datasetGTS = {'annotations' : []}
datasetRES = {'annotations' : []}

f = 0

l_a = []
l_p = []


def push_back(l, idx, val):
    if val == None:
        l += [{u'image_id' : idx, u'caption' : ''}]
        return l

    if type(val).__name__ == 'str':
        l += [{u'image_id' : idx, u'caption' : val}]
        return l

    if type(val).__name__ == 'list':

        if len(val) == 0:
            l += [{u'image_id' : idx, u'caption' : ''}]
            return l

        if type(val[0]).__name__ == 'str':
            s = ''
            for it in val:
                s += ' ' + it
            l += [{u'image_id' : idx, u'caption' : s}]
            return l
            
        if type(val[0]).__name__ == 'list':

            for tmp in val:

                s = ''
                for it in tmp:
                    s += ' ' + it
                l += [{u'image_id' : idx, u'caption' : s}]
            
            return l
            

for i in range(num_predictions):
    l_a = push_back(l_a, i, actual[i])
    l_p = push_back(l_p, i, predicted[i])


datasetGTS['annotations'] = l_a
datasetRES['annotations'] = l_p

"""print(datasetGTS)
print(datasetRES)"""

'print(datasetGTS)\nprint(datasetRES)'

In [ ]:
for metric in evaluation:
    print(metric + "  Score  =  ", evaluation[metric])

Bleu_1  Score  =   0.7951192609953405
Bleu_2  Score  =   0.6784421921698088
Bleu_3  Score  =   0.5867660843028581
Bleu_4  Score  =   0.491239665712838
METEOR  Score  =   0.3456371489074272
ROUGE_L  Score  =   0.7234715763122592
CIDEr  Score  =   0.8583709507484677


In [ ]:
evaluation = calculate_metrics(rng,datasetGTS,datasetRES)
print(evaluation)

tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 8333, 'reflen': 8171, 'guess': [8333, 7133, 5933, 4733], 'correct': [5505, 2513, 1354, 545]}
ratio: 1.0198262146614832
Bleu_1: 0.661
Bleu_2: 0.482
Bleu_3: 0.376
Bleu_4: 0.280
computing METEOR score...
METEOR: 0.256
computing Rouge score...
ROUGE_L: 0.607
computing CIDEr score...
CIDEr: 0.374
{'Bleu_1': 0.6606264250569229, 'Bleu_2': 0.4824342171648527, 'Bleu_3': 0.37590102276129944, 'Bleu_4': 0.27965341684527334, 'METEOR': 0.25613917809585407, 'ROUGE_L': 0.6068507553618968, 'CIDEr': 0.3739325644512447}


tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 7253, 'reflen': 7231, 'guess': [7253, 6053, 4853, 3653], 'correct': [5767, 3504, 2130, 1053]}
ratio: 1.0030424560916882
Bleu_1: 0.795
Bleu_2: 0.678
Bleu_3: 0.587
Bleu_4: 0.491
computing METEOR score...
METEOR: 0.346
computing Rouge score...
ROUGE_L: 0.723
computing CIDEr score...
CIDEr: 0.858
{'Bleu_1': 0.7951192609953405, 'Bleu_2': 0.6784421921698088, 'Bleu_3': 0.5867660843028581, 'Bleu_4': 0.491239665712838, 'METEOR': 0.3456371489074272, 'ROUGE_L': 0.7234715763122592, 'CIDEr': 0.8583709507484677}
